In [1]:
import sys
sys.path.insert(0,'./DCGAN-tensorflow')

import os
import scipy.misc
import numpy as np

from model import DCGAN
from utils import pp, visualize, to_json, show_all_variables, save_images

import tensorflow as tf


In [2]:
input_width, input_height = 128, 128
output_width, output_height = 128, 128
batch_size = 64
z_dim = 100
dataset_name = 'landscape_128_128'

run_config = tf.ConfigProto()
run_config.gpu_options.allow_growth=True

with tf.Session(config=run_config) as sess:
  dcgan = DCGAN(
      sess,
      input_width=input_width,
      input_height=input_height,
      output_width=output_width,
      output_height=output_height,
      batch_size=batch_size,
      sample_num=batch_size,
      z_dim=z_dim,
      dataset_name=dataset_name,
      input_fname_pattern='*.jpg',
      crop=False,
      checkpoint_dir='./DCGAN-tensorflow/checkpoint',
      sample_dir='.')


In [3]:

#dcgan.load('./DCGAN-tensorflow/checkpoint')
dcgan.load(checkpoint_dir='./DCGAN-tensorflow/checkpoint')

    

 [*] Reading checkpoints...
INFO:tensorflow:Restoring parameters from ./DCGAN-tensorflow/checkpoint/landscape_128_128_64_128_128/DCGAN.model-26002
 [*] Success to read DCGAN.model-26002


(True, 26002)

In [47]:
from __future__ import division
import math
import json
import random
import pprint
import scipy.misc
import numpy as np
from time import gmtime, strftime
from six.moves import xrange

import tensorflow as tf
import tensorflow.contrib.slim as slim

image_frame_dim = int(math.ceil(batch_size**.5))
z_sample = np.random.uniform(-1.0, 1.0, size=(batch_size, dcgan.z_dim))



def sample_progression(dcgan, n):
    z1 = np.random.uniform(-1, 1, size=(dcgan.z_dim))
    samples = []
    for i in range(n):
        z2 = np.random.uniform(-1, 1, size=(dcgan.z_dim))
        zs = np.array([ (1.0-t)*z1 + t*z2  for t in np.linspace(0, 1, dcgan.batch_size)])
        samples.append(sess.run(dcgan.sampler, feed_dict={dcgan.z: zs}))
        z1 = z2
    return samples
        
        
#z1 = np.random.uniform(-1, 1, size=(dcgan.z_dim))
#z2 = np.random.uniform(-1, 1, size=(dcgan.z_dim))
#z_sample = np.array([ (1.0-t)*z1 + t*z2  for t in np.linspace(0, 1, batch_size)])

#samples = sess.run(dcgan.sampler, feed_dict={dcgan.z: z_sample})

samples= sample_progression(dcgan, 5)
#save_images(samples, [image_frame_dim, image_frame_dim], './samples/test_%s.png' % strftime("%Y-%m-%d-%H-%M-%S", gmtime()))

samples = [s for segment in samples for s in segment]

print(len(samples))

320


In [35]:
save_images(samples, [image_frame_dim, image_frame_dim], './samples/test_%s.png' % strftime("%Y-%m-%d-%H-%M-%S", gmtime()))

In [51]:
#samples[0]

import moviepy.editor as mpy

def make_frame(t):
    nf, fps = 320, 30
    x = samples[int(min(nf-1, fps*t))]
    return ((x+1)/2*255).astype(np.uint8)

clip = mpy.VideoClip(make_frame, duration=10.6667)
clip.write_videofile('test5.mp4', fps=30)

[MoviePy] >>>> Building video test5.mp4
[MoviePy] Writing video test5.mp4







  0%|          | 0/321 [00:00<?, ?it/s]




 16%|█▌        | 51/321 [00:00<00:00, 504.74it/s]




 51%|█████     | 164/321 [00:00<00:00, 605.15it/s]




 87%|████████▋ | 278/321 [00:00<00:00, 703.41it/s]




100%|██████████| 321/321 [00:00<00:00, 944.26it/s]

[MoviePy] Done.
[MoviePy] >>>> Video ready: test5.mp4 



In [18]:
samples[0].astype('uint8').shape

(128, 128, 3)